# THE SPARK FOUNDATION #GRIPJUNE23

# Detection of Face mask-Task 4 (Computer vision and IOT)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
!pip install imutils

In [ ]:
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

## Initial Hyperparameters

In [ ]:
init_learning_rate = 1e-4
epochs = 20
batch_size = 32

In [ ]:
directory = r"D:\FaceMaskDetection-main\FaceMaskDetection-main\dataset"
categories = ["with_mask","without_mask"]

In [ ]:
data =[]
labels =[]
for category in categories:
    path = os.path.join(directory,category)
    for img in os.listdir(path):
        img_path = os.path.join(path,img)
        image = load_img(img_path,target_size=(224,224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(category)

## one hot encoding the labels

In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
data = np.array(data,dtype="float32")

In [ ]:
labels = np.array(labels)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data,labels,test_size=0.2,stratify=labels,random_state=42)

## constructing image generation for augmentation

In [ ]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range =0.2,
    height_shift_range = 0.2,
    shear_range=0.15,
    horizontal_flip = True,
    fill_mode="nearest"
)

## loading the MobileNetV2 network

In [ ]:
basemodel = MobileNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))# 3 for rgb format

In [ ]:
# Top of base model will be head of the model
from tensorflow.keras.layers import Flatten
headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size=(7,7))(headmodel)
headmodel = Flatten(name= "flatten")(headmodel)
headmodel = Dense(128,activation="relu")(headmodel)
headmodel = Dropout(0.5)(headmodel)
headmodel = Dense(2,activation="softmax")(headmodel)

In [ ]:
model = Model(inputs=basemodel.input,outputs=headmodel)

In [ ]:
# Freezing the base layers of network, the weights of these layers will not be updated during backpropagation, whereas head wights will change
for layer in basemodel.layers:
    layer.trainable = False

In [ ]:
#compile the model
#opt = Adam(learning_rate=init_learning_rate,leg=init_learning_rate/epochs)
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=["accuracy"])

In [ ]:
# training head of network
H = model.fit(
    aug.flow(x_train,y_train,batch_size=batch_size),
    steps_per_epoch=len(x_train) //batch_size,
    validation_data = (x_test,y_test),
    validation_steps=len(x_test)//batch_size,
    epochs=epochs
)

In [ ]:
predictions = model.predict(x_test,batch_size=batch_size)

In [ ]:
predictions = np.argmax(predictions,axis=1)

In [ ]:
predictions

In [ ]:
# classification report
print(classification_report(y_test.argmax(axis=1),predictions,target_names=lb.classes_))

In [ ]:
model.save("Mask_detector.model",save_format="h5")

In [ ]:
# Accuracy of the model
n = epochs
plt.figure(figsize=(5,5))
plt.plot(np.arange(0,n),H.history["loss"],label="train_loss")
plt.plot(np.arange(0,n),H.history["val_loss"],label="val_loss")
plt.plot(np.arange(0,n),H.history["accuracy"],label="train_accuracy")
plt.plot(np.arange(0,n),H.history["val_accuracy"],label="val_accuracy")
plt.title("Training Loss and Accuracy")
plt.xlabel("No. of epochs")
plt.ylabel("Loss/accuracy")
plt.legend(loc="lower left")
plt.show()

### Video Streaming

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [ ]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

def detect_and_predict_mask(frame, faceNet, maskNet):
# grab the dimensions of the frame and then construct a blob
# from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
        (104.0, 177.0, 123.0))

# pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

# initialize our list of faces, their corresponding locations,
# and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []

# loop over the detections
    for i in range(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with
    # the detection
        confidence = detections[0, 0, i, 2]

# filter out weak detections by ensuring the confidence is
# greater than the minimum confidence
        if confidence > 0.5:
# compute the (x, y)-coordinates of the bounding box for
# the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
# ensure the bounding boxes fall within the dimensions of
# the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
           
          
            
            
      
           

# extract the face ROI, convert it from BGR to RGB channel
# ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
# add the face and bounding boxes to their respective
# lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))
            
            
            

# only make a predictions if at least one face was detected
    if len(faces) > 0:
# for faster inference we'll make batch predictions on *all*
# faces at the same time rather than one-by-one predictions
# in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
# return a 2-tuple of the face locations and their corresponding
# locations
    return (locs, preds)

# load our serialized face detector model from disk
prototxtPath = r"D:\FaceMaskDetection-main\FaceMaskDetection-main\face_detector\deploy.prototxt"
weightsPath = r"D:\FaceMaskDetection-main\FaceMaskDetection-main\face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
maskNet = load_model("Mask_detector.model")

# initialize the video stream
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()

# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
# unpack the bounding box and predictions
            (startX, startY, endX, endY) = box
            (mask, withoutMask) = pred
# determine the class label and color we'll use to draw
# the bounding box and text
            label = "Mask" if mask > withoutMask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

# include the probability in the label
            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
# display the label and bounding box rectangle on the output
# frame
            cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
# show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

# if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()